## Gregory M. Eirich
## Example 
## Lab #6

# Lab 6 Examples


## 1. Run a naive ("pooled") OLS regression on the panel data.  Tell we how you expect your Xs to affect your Y and why.   Interpret your results.

## 2. Run a first differences regression on the same model in Question 1.  Interpret your results.  Do you draw a different conclusion than in Question 1?  Explain.


In [1]:
pip install linearmodels

    100% |████████████████████████████████| 1.5MB 2.7MB/s ta 0:00:01
    100% |████████████████████████████████| 1.9MB 2.2MB/s ta 0:00:01
    100% |████████████████████████████████| 17.5MB 2.3MB/s ta 0:00:011
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Stored in directory: /Users/gregoryeirich/Library/Caches/pip/wheels/b8/11/b9/55a5302e1fd2b41f3d781d12830dee2a7259f3d61b5d80f93b
Successfully built statsmodels
  Found existing installation: Cython 0.29.6
    Uninstalling Cython-0.29.6:
      Successfully uninstalled Cython-0.29.6
  Found existing installation: statsmodels 0.9.0
    Uninstalling statsmodels-0.9.0:
      Successfully uninstalled statsmodels-0.9.0
Note: you may need to restart the kernel to use updated packages.


In [2]:
from __future__ import division
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import os
import matplotlib.pyplot as plt
import linearmodels as lm

from linearmodels import PanelOLS
from linearmodels import RandomEffects
from linearmodels import FirstDifferenceOLS

## Use the GSS Panel in Folder for Lab 6 on Courseworks.

In [5]:
os.chdir('/Users/gregoryeirich/Desktop/Desktop/Data Analysis') # change working directory

g = pd.read_csv("panel-for-R.csv.xls") ## choose "panel-for-R.csv" and more information on variables are here: http://sda.berkeley.edu/sdaweb/analysis/?dataset=gss06panelw3 ##

g.head()

,idnum,panelwave,ballot,form,formwt,oversamp,sample,panstat_2,panstat_3,mar1,...,wtpan12,wtpan123,wtpannr12,wtpannr123,xmarsex,xmovie,xnorcsiz,year,yearval,zodiac
0,9,1,3,2,1,1,9,1,1,5.0,...,0.414689,0.487828,0.435503,0.470575,1.0,2.0,1.0,2006.0,NaN,9.0
1,9,2,3,2,1,1,9,1,1,5.0,...,0.414689,0.487828,0.435503,0.470575,1.0,1.0,1.0,2008.0,NaN,9.0
2,9,3,3,2,1,1,9,1,1,1.0,...,0.414689,0.487828,0.435503,0.470575,1.0,1.0,1.0,2010.0,NaN,9.0
3,10,1,1,1,1,1,9,1,1,5.0,...,0.829377,0.858741,0.766632,0.828371,1.0,NaN,1.0,2006.0,NaN,3.0
4,10,2,1,1,1,1,9,1,1,5.0,...,0.829377,0.858741,0.766632,0.828371,1.0,NaN,1.0,2008.0,NaN,3.0


## We are going to revisit an example earlier in the semester: the relationship between hours worked and salary earned.  There was a positive relationship between hours worked and money earned, but we weren't sure why.   Before we run our regressions, we should log both hours and income.

In [20]:
g['lnrealrinc'] = np.log(g['realrinc']) 
g['lnhrs1'] = np.log(g['hrs1']) 

In [21]:
g[["lnrealrinc", "lnhrs1"]].describe()

,lnrealrinc,lnhrs1
count,2847.000000,2802.000000
mean,9.592979,3.649706
std,1.162260,0.479530
min,5.556828,0.000000
25%,9.131094,3.637586
50%,9.824241,3.688879
75%,10.290534,3.912023
max,13.081842,4.488636


## 1. Run a naive ("pooled") OLS regression on the panel data.  Tell we how you expect your Xs to affect your Y and why.   Interpret your results.

## The question is how are income and hours worked related?

As you can see, in the simple OLS, it appears that for every hour more worked, on average, $614 more in income is earned.  It is quite statistically significant and it explains about 3 percent of the variance in salary earned

In [6]:
lm_ols = smf.ols(formula = 'realrinc ~ hrs1 ', data = g).fit()
print (lm_ols.summary())

                            OLS Regression Results                            
Dep. Variable:               realrinc   R-squared:                       0.029
Model:                            OLS   Adj. R-squared:                  0.028
Method:                 Least Squares   F-statistic:                     71.21
Date:                Tue, 18 May 2021   Prob (F-statistic):           5.48e-17
Time:                        20:18:50   Log-Likelihood:                -29349.
No. Observations:                2397   AIC:                         5.870e+04
Df Residuals:                    2395   BIC:                         5.871e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   2361.4863   3248.606      0.727      0.4

## What if we run the same model as an elasticity, log-log model?  We regress log hours on log  income.

This is also quite statistically significant, where for every 1% increase in hours worked, on average, we see a .83% increase in money.  Look at that increase in R-sq, now explaining almost 14% of the variance in log salary.

In [22]:
lm_ols2 = smf.ols(formula = 'lnrealrinc ~ lnhrs1 ', data = g).fit()
print (lm_ols2.summary())

                            OLS Regression Results                            
Dep. Variable:             lnrealrinc   R-squared:                       0.137
Model:                            OLS   Adj. R-squared:                  0.137
Method:                 Least Squares   F-statistic:                     380.5
Date:                Tue, 18 May 2021   Prob (F-statistic):           9.31e-79
Time:                        20:42:08   Log-Likelihood:                -3339.7
No. Observations:                2397   AIC:                             6683.
Df Residuals:                    2395   BIC:                             6695.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      6.6546      0.159     41.859      0.0

## 2. Run a first differences regression on the same model in Question 1.  Interpret your results.  Do you draw a different conclusion than in Question 1?  Explain.

Let's start by regressing changes in hours worked to changes in salary.  We see that there appears to be no association between changes in hours and changes in salary (B = essentially zero).  The t-statistic is not statistical significant.

In [7]:
g = g.set_index(['idnum', 'panelwave'])
lm_fd = FirstDifferenceOLS.from_formula('realrinc ~ hrs1', g)
res_fd = lm_fd.fit(cov_type='clustered', cluster_entity=True)
print(res_fd)

                     FirstDifferenceOLS Estimation Summary                      
Dep. Variable:               realrinc   R-squared:                     7.907e-06
Estimator:         FirstDifferenceOLS   R-squared (Between):             -0.0151
No. Observations:                1107   R-squared (Within):              -0.0002
Date:                Tue, May 18 2021   R-squared (Overall):             -0.0122
Time:                        20:19:21   Log-likelihood                -1.391e+04
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      0.0087
Entities:                        1226   P-value                           0.9255
Avg Obs:                       1.9551   Distribution:                  F(1,1106)
Min Obs:                       1.0000                                           
Max Obs:                       3.0000   F-statistic (robust):             0.0065
                            

/Users/gregoryeirich/anaconda3/lib/python3.7/site-packages/linearmodels/shared/exceptions.py:35: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


## How about a first difference model with logged income and logged hours?

Now, the relationship between changes in log hours and log income is now marginally statistically signicant (p<.1) and it is in the expected direction: for each 1% change in hours, there is a 0.146% change in salary.  Also, the (within) R-sq shows that these changes explain about 1.38% in the variance in logged salary.

In [23]:
lm_fd2 = FirstDifferenceOLS.from_formula('lnrealrinc ~ lnhrs1', g)
res_fd2 = lm_fd2.fit(cov_type='clustered', cluster_entity=True)
print(res_fd2)

                     FirstDifferenceOLS Estimation Summary                      
Dep. Variable:             lnrealrinc   R-squared:                        0.0054
Estimator:         FirstDifferenceOLS   R-squared (Between):              0.1068
No. Observations:                1107   R-squared (Within):               0.0138
Date:                Tue, May 18 2021   R-squared (Overall):              0.1065
Time:                        20:43:12   Log-likelihood                   -1327.1
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      5.9790
Entities:                        1226   P-value                           0.0146
Avg Obs:                       1.9551   Distribution:                  F(1,1106)
Min Obs:                       1.0000                                           
Max Obs:                       3.0000   F-statistic (robust):             3.8067
                            

/Users/gregoryeirich/anaconda3/lib/python3.7/site-packages/linearmodels/shared/exceptions.py:35: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)
